In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import sys

sys.path.append(os.path.abspath("../"))
from dataset import DS_NAMES
from dataset import DS_DIMENSIONS
from utilities.figure_tools import get_latest_results_directory, make_fig_directory, load_results

cmap = plt.get_cmap("Set1")
colors = cmap.colors

results_path = "../results/turing/vary_ntrees.csv"
output_dir = "./vary-ntrees/"
export_figures = True

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [2]:
all_results = pd.read_csv(results_path).groupby(["dataset", "explainer", "n_trees"]).mean().reset_index()
print("explainers:", list(all_results["explainer"].unique()))
print("datasets", list(all_results["dataset"].unique()))
all_results.head()

explainers: ['FACETIndex', 'OCEAN']
datasets ['cancer', 'glass', 'magic', 'spambase', 'vertebral']


,dataset,explainer,n_trees,iteration,max_depth,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,FACETIndex,10,2,5,0.933333,0.937627,0.869294,0.901477,1.0,0.180331,5.04,6.427461,0.646048,0.032302,20
1,cancer,FACETIndex,50,2,5,0.943860,0.942500,0.895361,0.918105,1.0,0.301897,10.32,23.674444,2.564363,0.128218,20
2,cancer,FACETIndex,100,2,5,0.947368,0.948096,0.901243,0.924017,1.0,0.371090,13.59,37.822390,3.421698,0.171085,20
3,cancer,FACETIndex,200,2,5,0.949123,0.957651,0.895361,0.925283,1.0,0.423372,16.97,65.855959,4.064293,0.203215,20
4,cancer,FACETIndex,300,2,5,0.945614,0.948848,0.895361,0.920982,1.0,0.449443,18.50,95.861910,4.286393,0.214320,20


In [3]:
idx_match = all_results["n_trees"] == 10
all_results[idx_match].to_csv("temp_table.csv", index=False)

In [4]:
cmap = plt.get_cmap("tab10")
colors = cmap.colors
ds_colors = {
    "cancer": colors[0],
    "glass": colors[1],
    "magic": colors[2],
    "spambase": colors[3],
    "vertebral": colors[4]
}
epl_colors = {
    "AFT": colors[0],
    "FACETIndex": colors[1],
    "OCEAN": colors[2],
    "RFOCSE": colors[3],
    "MACE": colors[4]
}
explainer_markers = {
    "AFT": "o",
    "FACETIndex": "^",
    "OCEAN": "s",
    "RFOCSE": "v",
    "MACE": "D"
}
marker_size = 8
line_width = 2
marker_sizes = {
    "AFT": marker_size,
    "FACETIndex": marker_size,
    "OCEAN": marker_size,
    "RFOCSE": marker_size,
    "MACE": marker_size
}

In [5]:
def render_plot(xfeature, yfeature, xlabel, ylabel, title, save_name=None, xrange=None, yrange=None, xlog=False, lloc=None, ylog=False):

    for ds in all_results["dataset"].unique():
        plt.figure()
        is_ds = all_results["dataset"] == ds
        for expl in all_results["explainer"].unique():
            is_expl = all_results["explainer"] == expl
            matching_rows = is_ds & is_expl
            plt.plot(all_results[matching_rows][xfeature], all_results[matching_rows][yfeature], label=expl, marker=explainer_markers[expl], lw=line_width, ms=marker_sizes[expl])

        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(ds)
        plt.legend(loc="best")
        if ylog:
            plt.yscale('log')
        if xrange is not None:
            plt.xlim(xrange)
        if yrange is not None:
            plt.ylim(yrange)
        if export_figures and save_name is not None:
            plt.savefig(output_dir + ds + "_" + save_name, bbox_inches="tight")

In [6]:
df = all_results[all_results["n_trees"] == 100]
df_ocean = df[df["explainer"] == "OCEAN"]
df_facet = df[df["explainer"] == "FACETIndex"]
ocean_dist = df_ocean["avg_dist"].to_numpy()
facet_dist = df_facet["avg_dist"].to_numpy()

print(df_facet["dataset"].to_numpy())
to_ocean = facet_dist / ocean_dist
print(to_ocean)
prev_results = [2.28858939, 2.0159912, 1.87535734, 10.16738057, 1.46097089]
print(prev_results) # untuned values
print(to_ocean / prev_results)

['cancer' 'glass' 'magic' 'spambase' 'vertebral']
[1.7841393  2.87533551 1.59169692 6.36873142 1.29994021]
[2.28858939, 2.0159912, 1.87535734, 10.16738057, 1.46097089]
[0.77958034 1.42626392 0.84874327 0.62638861 0.88977831]


In [7]:
render_plot("n_trees", "avg_dist", "Num Trees", "Distance", "Average Distance vs Num Trees", save_name="dist_vs_trees.png", yrange=[0,1])

In [8]:
render_plot("n_trees", "per_valid", "Num Trees", "Percent Valid Examples", "Percent Valid Examples vs Num Trees", save_name="valid_vs_trees.png")

In [9]:
# prep time vs number of trees
render_plot("n_trees", "prep_time", "Num Trees", "Preptime (sec)", "Prep Time vs Num Trees", save_name="prep_vs_trees.png")

In [10]:
# sample time vs number of trees
render_plot("n_trees", "sample_time", "Num Trees", "Sample Runtime (sec)", "Explanation Time vs Num Trees", save_name=None, ylog=False) # "sampletime_vs_trees.png"